**Import Data User**

In [77]:
# !unzip yelp_academic_dataset_user.json.zip

In [2]:
!rm yelp_academic_dataset_user.json.zip
!ls

derby.log     ondemand		      yelp_academic_dataset_user.json
metastore_db  spark_neighbours.ipynb


In [3]:
a = spark.read.json("yelp_academic_dataset_user.json")

In [4]:
a.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)



**Extract Friends**

In [5]:
friends = a.rdd.map(lambda row: (row['user_id'], set(row['friends'].split(', '))))

In [6]:
import pandas as pd

In [7]:
path = "yelp_academic_dataset_user.json"
size = 1
user = pd.read_json(path, lines=True, chunksize=size)

In [8]:
df = None
for u in user:
    df = u
    break

In [9]:
sample_user_id = set(df.user_id)

In [10]:
all_user = sample_user_id

**K-HOP with BFS**

In [11]:
for _ in range(2):
    sample_user_id = friends.filter(lambda x: x[0] in sample_user_id).map(lambda row: row[1]).reduce(lambda x, y: x | y) - all_user
    print(len(sample_user_id))
    all_user |= sample_user_id

14995
461105


In [12]:
len(all_user)

476101

In [31]:
friends.map(lambda row: (1, len(row[1]))).reduce(lambda x, y: (x[0]+y[0], x[1]+y[1]))

(1987897, 106104025)

Exception in thread "serve RDD 17" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)


In [13]:
106104025/1987897

53.375011381374385

In [14]:
with open("user_id.txt", "w+") as f:
    for u in all_user:
        f.write(u+"\n")

In [15]:
cnt = 0
for i in all_user:
    print(i)
    cnt += 1
    if cnt >= 5:
        break

u9fOD3y2BxYifsyXQZMSTw
GnAmnY_g3hHzVOCCANk7Jg
eFf6AQ3kIRzSg_uRBzlz-Q
GUAzqV1Pwf7mGaZT18okFg
4ciwJaCgt4JbXlC6v1PcPQ


In [16]:
!head user_id.txt

u9fOD3y2BxYifsyXQZMSTw
GnAmnY_g3hHzVOCCANk7Jg
eFf6AQ3kIRzSg_uRBzlz-Q
GUAzqV1Pwf7mGaZT18okFg
4ciwJaCgt4JbXlC6v1PcPQ
3nuvHQPDQnH-_HTOT0hxqw
31ouhXIdHOHIgZpexUMXNA
3Z5ksD1H17Sla3F271cltw
M4XmTkBMWVfeO0kJ4hdFPw
H4iTSKSvDHW-LJ1hZxSRmg


In [21]:
u_df = a.rdd.filter(lambda x: x['user_id'] in all_user)

In [22]:
user_df = u_df.toDF()

In [23]:
user_df.count()

67095

**Store Data**

In [33]:
user_df.write.csv("sample_user", sep="\t")

In [34]:
!cat sample_user/*.csv > sample_user.csv

In [36]:
sample_user = pd.read_csv("sample_user.csv", header=None, sep="\t")

In [38]:
sample_user.columns = a.columns

In [41]:
sample_user.to_csv("sample_user.csv", sep="\t")

**Import Data Review**

In [42]:
!unzip yelp_academic_dataset_review.json.zip
!rm yelp_academic_dataset_review.json.zip

Archive:  yelp_academic_dataset_review.json.zip
  inflating: yelp_academic_dataset_review.json  


In [57]:
b = spark.read.json("yelp_academic_dataset_review.json")

In [58]:
b.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [59]:
review = b.rdd.filter(lambda row: row['user_id'] in all_user)

In [83]:
review_df = review.toDF().toPandas()

In [86]:
review_df.drop(columns=['review_id', 'text'], inplace=True)

In [87]:
review_df.to_csv("sample_review.csv", sep="\t")

In [88]:
real_a = pd.read_csv("sample_user.csv", sep="\t")
real_b = pd.read_csv("sample_review.csv", sep="\t")

In [89]:
real_a.shape, real_b.shape

((67095, 23), (883363, 8))

In [90]:
real_b.columns

Index(['Unnamed: 0', 'business_id', 'cool', 'date', 'funny', 'stars', 'useful',
       'user_id'],
      dtype='object')